# Income Classification

Original Dataset: https://archive.ics.uci.edu/ml/datasets/adult

### Imports

In [5]:
import pandas as pd

# Preparation

## Load data

In [30]:
train_df = pd.read_csv('adult.data', delimiter=',', skipinitialspace=True)
test_df = pd.read_csv('adult.test', delimiter=',', skipinitialspace=True)

train_df_size = train_df.shape[0]
test_df_size = test_df.shape[0]
df_size = train_df_size + test_df_size

print(f'Train Dataset Size: {train_df_size} ({(train_df_size/df_size):.2%})')
print(f'Test Dataset Size: {test_df_size} ({(test_df_size/df_size):.2%})')

Train Dataset Size: 32561 (66.67%)
Test Dataset Size: 16281 (33.33%)


## Pre Processing

First we need to label the data with ones for income >50k and zeros for income <=50k.

In [32]:
train_df['income'] = train_df['income'].apply(lambda x: 1 if x == '>50K' else 0)
test_df['income'] = test_df['income'].apply(lambda x: 1 if x == '>50K' else 0)